In [ ]:
# !pip install -e ..

In [ ]:
import collections
import datetime
import pathlib

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from assistance.api.keys import get_notion_api_key

In [ ]:
API_KEY = get_notion_api_key()
PARENT_PAGE_ID = "ad816892782d478d9998f700a5c783be"

In [ ]:
import aiohttp

In [ ]:
async def get_all_sub_blocks(block_id):
    parameters = ""
    has_more = True
    results = []
    
    async with aiohttp.ClientSession() as session:
        while True:
            url = f"https://api.notion.com/v1/blocks/{block_id}/children{parameters}"

            headers = {
                "accept": "application/json",
                "Notion-Version": "2022-06-28",
                "Authorization": f"Bearer {API_KEY}",
            }

            async with session.get(url, headers=headers) as resp:
                json = await resp.json()

            results += json['results']
            has_more = json['has_more']

            if not has_more:
                return results

            parameters = f"?start_cursor={json['next_cursor']}"

In [ ]:
async with aiohttp.ClientSession() as session:
    url = f"https://api.notion.com/v1/blocks/e99fc620-ec88-4a6b-bf40-1e1e0dbb27f1/children"

    headers = {
        "accept": "application/json",
        "Notion-Version": "2022-06-28",
        "Authorization": f"Bearer {API_KEY}",
    }

    async with session.get(url, headers=headers) as resp:
        json = await resp.json()

In [ ]:
a_page = await get_all_sub_blocks("e99fc620-ec88-4a6b-bf40-1e1e0dbb27f1")

In [ ]:
# a_page

In [ ]:
transcript_items = []

for item in a_page:
    for rich_text in item['paragraph']['rich_text']:
        transcript_items.append(rich_text['plain_text'])

transcript = "\n\n".join(transcript_items)

print(transcript)

In [ ]:
paragraphs

In [ ]:
results = await get_all_sub_blocks()

In [ ]:
pages = {
    item['id']: {'title': item['child_page']['title'], 'created_time': item['created_time']}
    for item in results
    if item['type'] == 'child_page'
}

In [ ]:
records_by_time = collections.defaultdict(list)

for key, item in pages.items():
    records_by_time[item['title']].append((datetime.datetime.strptime(item['created_time'], '%Y-%m-%dT%H:%M:%S.%fZ'), key))
    
for key, item in records_by_time.items():
    records_by_time[key] = sorted(records_by_time[key])

In [ ]:
transcripts_dir = pathlib.Path.home() / ".assistance.chat" / "transcripts"

In [ ]:
for page_id, details in pages.items():
    user_id = details['title']
    timestamp = details['created_time']
    
    user_dir = transcripts_dir / user_id
    user_dir.mkdir(exist_ok=True, parents=True)
    
    filename = timestamp.replace("-", "").replace(":", "").replace(".", "_")
    
    with open(filename, "w") as f:
        

In [ ]:
pages

In [ ]:
pages

In [ ]:
results